<a href="https://colab.research.google.com/github/Yoosofbidardel/Facial-Exprssion-Recognition-using-keras/blob/main/facial_expression3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # Second and Third approahces
 
  here we are going to use batches of data each time in order to prevent ram crashig
  

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
Train_dir = "/content/drive/MyDrive/bachelor_project/train/"
Test_dir = "/content/drive/MyDrive/bachelor_project/test/"

Classes  = ['angry','disgust','fear','happy','neutral','sad','surprise']

In [ ]:
#just a simple test to see what we are facing 
image_gen= ImageDataGenerator(rescale=1/255)
#image_gen_test.flow_from_directory(Data_dir)

# now it is better to put dataset into batches


In [ ]:
#initialization
image_size = (224,224) # input of imageNEt 224*224
batch_size= 32  # trial and error

train_img_gen= image_gen.flow_from_directory(Train_dir,target_size= image_size,batch_size= batch_size, class_mode= 'sparse')


Found 28729 images belonging to 7 classes.


In [ ]:
len(train_img_gen)
train_img_gen

In [ ]:
test_img_gen= image_gen.flow_from_directory(Test_dir,target_size= image_size,batch_size= batch_size, class_mode= 'sparse')


Found 7178 images belonging to 7 classes.


In [ ]:
train_img_gen.class_indices

{'angry': 0,
 'disgust': 1,
 'fear': 2,
 'happy': 3,
 'neutral': 4,
 'sad': 5,
 'surprise': 6}

In [ ]:
test_img_gen.labels.shape



(7178,)

In [ ]:
my_model = tf.keras.applications.MobileNetV2() # pretrained model

14548992/14536120 [==============================] - 0s 0us/step


In [ ]:
my_model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                              

# we have to freeze the first and the last layer to modify our own model


In [ ]:
input_base= my_model.layers[0].input   

In [ ]:
output_base= my_model.layers[-2].output   # we eliminated the last layer

In [ ]:
final_output= layers.Dense(128)(output_base)
final_output= layers.Activation("relu")(final_output)
final_output= layers.Dense(64)(final_output)
final_output= layers.Activation("relu")(final_output)
final_output= layers.Dense(7,activation='softmax')(final_output)





In [ ]:
final_output

<KerasTensor: shape=(None, 7) dtype=float32 (created by layer 'dense_2')>

In [ ]:
new_model= keras.Model(inputs=input_base,outputs=final_output)

In [ ]:
new_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                             

In [ ]:
checkpoint_path = "/content/drive/MyDrive/bachelor_project/training_3/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 32
step_per_epoch = 200

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=2*step_per_epoch)

In [ ]:
os.listdir(checkpoint_dir)

['cp-0010.ckpt.data-00000-of-00001',
 'cp-0010.ckpt.index',
 'cp-0000.ckpt.index',
 'cp-0000.ckpt.data-00000-of-00001',
 'cp-0002.ckpt.index',
 'cp-0002.ckpt.data-00000-of-00001',
 'cp-0004.ckpt.index',
 'cp-0004.ckpt.data-00000-of-00001',
 'cp-0005.ckpt.index',
 'cp-0005.ckpt.data-00000-of-00001',
 'cp-0007.ckpt.index',
 'cp-0007.ckpt.data-00000-of-00001',
 'cp-0008.ckpt.index',
 'checkpoint',
 'cp-0008.ckpt.data-00000-of-00001']

In [ ]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'/content/drive/MyDrive/bachelor_project/training_2/cp-0008.ckpt'

In [ ]:
new_model.compile(loss='sparse_categorical_crossentropy', optimizer = "adam", metrics = ["accuracy"])

In [ ]:
new_model.load_weights(latest)

In [ ]:
new_model.save_weights(checkpoint_path.format(epoch=0))

In [ ]:
results = new_model.fit_generator(train_img_gen,epochs=25,steps_per_epoch=step_per_epoch,validation_data= test_img_gen, callbacks=[cp_callback])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """Entry point for launching an IPython kernel.


Epoch 1/25
100/100 [==============================] - 1822s 18s/step - loss: 1.1453 - accuracy: 0.5766 - val_loss: 4.9462 - val_accuracy: 0.2648
Epoch 2/25
 59/100 [================>.............] - ETA: 2:56 - loss: 1.0799 - accuracy: 0.6043
Epoch 00002: saving model to /content/drive/MyDrive/bachelor_project/training_2/cp-0002.ckpt
100/100 [==============================] - 449s 5s/step - loss: 1.1142 - accuracy: 0.5875 - val_loss: 4.6427 - val_accuracy: 0.3240
Epoch 3/25
100/100 [==============================] - 420s 4s/step - loss: 1.1016 - accuracy: 0.5878 - val_loss: 3.3165 - val_accuracy: 0.4440
Epoch 4/25
 19/100 [====>.........................] - ETA: 5:04 - loss: 1.1523 - accuracy: 0.5691
Epoch 00004: saving model to /content/drive/MyDrive/bachelor_project/training_2/cp-0004.ckpt
100/100 [==============================] - 393s 4s/step - loss: 1.1024 - accuracy: 0.5934 - val_loss: 2.9820 - val_accuracy: 0.4060
Epoch 5/25
 79/100 [======================>.......] - ETA: 1:04 - 

In [ ]:
results2 = new_model.fit_generator(train_img_gen,epochs=25,steps_per_epoch=step_per_epoch,validation_data= test_img_gen, callbacks=[cp_callback])
# 

In [ ]:
new_model.save('/content/drive/MyDrive/bachelor_project/my_model_12/26/2021.h5')

NameError: ignored